# Hodgkin-Huxley Model: Sinusoidal Current Stimulation

This notebook explores the membrane potential dynamics under sinusoidal current injection.

We'll investigate how different parameters of a sine wave current affect the neuronal response.

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import matplotlib.pyplot as plt
from src.model import HodgkinHuxleyModel
from src.visualization import plot_membrane_potential, plot_gate_variables

plt.style.use('seaborn-darkgrid')

## Sinusoidal Current Generator

We'll create a function to generate a sine wave current with configurable parameters:
- Amplitude: Peak current intensity
- Frequency: Oscillations per time unit
- Duration: Total stimulation time
- Phase: Optional phase shift

In [ ]:
def sine_current(t, amplitude=5, frequency=1, duration=20, phase=0):
    """Generate a sinusoidal current stimulus.
    
    Args:
        t (float): Current time point
        amplitude (float): Peak current amplitude (μA/cm²)
        frequency (float): Frequency of oscillation (Hz)
        duration (float): Total stimulation duration (ms)
        phase (float): Phase shift (radians)
    
    Returns:
        float: Current intensity at time t
    """
    return amplitude * np.sin(2 * np.pi * frequency * t / 1000 + phase) if t <= duration else 0

## Experiment 1: Low-Frequency Sinusoidal Stimulation

First, we'll simulate a low-frequency sine wave current with moderate amplitude.

In [ ]:
# Low-frequency simulation
model = HodgkinHuxleyModel()

# Parameters
amplitude_low = 5    # μA/cm²
frequency_low = 0.5  # 0.5 Hz
duration_low = 30    # 30 ms

# Simulate
t_span_low = [0, duration_low]
t_low, V_low, n_low, m_low, h_low = model.simulate(
    t_span_low, 
    I_ext_func=lambda t: sine_current(t, amplitude=amplitude_low, frequency=frequency_low)
)

# Generate current array for plotting
I_ext_low = np.array([sine_current(t_i, amplitude=amplitude_low, frequency=frequency_low) for t_i in t_low])

# Plot membrane potential and gate variables
fig1 = plot_membrane_potential(t_low, V_low, f"Low Frequency (0.5 Hz) Sinusoidal Current", I_ext_low)
plt.show()

fig2 = plot_gate_variables(t_low, n_low, m_low, h_low)
plt.show()

## Experiment 2: High-Frequency Sinusoidal Stimulation

Now we'll compare with a higher frequency sine wave current.

In [ ]:
# Reset model to initial conditions
model.reset_state()

# High-frequency parameters
amplitude_high = 5    # μA/cm²
frequency_high = 5    # 5 Hz
duration_high = 30    # 30 ms

# Simulate
t_span_high = [0, duration_high]
t_high, V_high, n_high, m_high, h_high = model.simulate(
    t_span_high, 
    I_ext_func=lambda t: sine_current(t, amplitude=amplitude_high, frequency=frequency_high)
)

# Generate current array for plotting
I_ext_high = np.array([sine_current(t_i, amplitude=amplitude_high, frequency=frequency_high) for t_i in t_high])

# Plot membrane potential and gate variables
fig3 = plot_membrane_potential(t_high, V_high, f"High Frequency (5 Hz) Sinusoidal Current", I_ext_high)
plt.show()

fig4 = plot_gate_variables(t_high, n_high, m_high, h_high)
plt.show()

## Experiment 3: Comparing Multiple Sinusoidal Stimuli

We'll create a subplot to visualize different sinusoidal current characteristics side by side.

In [ ]:
# Define parameter combinations
stimuli_params = [
    {"amplitude": 3, "frequency": 0.5, "label": "Low Amp, Low Freq"},
    {"amplitude": 5, "frequency": 1, "label": "Mod Amp, Mod Freq"},
    {"amplitude": 7, "frequency": 2, "label": "High Amp, High Freq"}
]

duration = 30  # ms
t_span = [0, duration]

fig, axes = plt.subplots(len(stimuli_params), 1, figsize=(10, 4*len(stimuli_params)))

for i, params in enumerate(stimuli_params):
    model.reset_state()
    t, V, n, m, h = model.simulate(
        t_span, 
        I_ext_func=lambda t: sine_current(t, **{k: params[k] for k in ['amplitude', 'frequency']})
    )
    
    # Plot membrane potential
    axes[i].plot(t, V, label='Membrane Potential')
    axes[i].set_title(params["label"])
    axes[i].set_ylabel('Voltage (mV)')
    axes[i].grid(True)
    
    # Overlay current
    ax2 = axes[i].twinx()
    current = [sine_current(t_i, **{k: params[k] for k in ['amplitude', 'frequency']}) for t_i in t]
    ax2.plot(t, current, color='red', linestyle='--', label='Current')
    ax2.set_ylabel('Current (μA/cm²)')

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.savefig('../data/results/sine_current_membrane_response.png')
plt.show()